## EVALUACION PARCIAL
### PROYECTO Mi-Bici
**TELLES AMEZCUA VICTOR MANUEL | 737066**

### **Importacion de librerias**
Importar librerias necesarias para el procesamiento, analisis y visualizacion de datos en el proyecto.

`pandas` = Para manipulacion y analisis de datos

`numpy` = Para calculos numericos y estructura de datos eficientes

`matplotlib` = Para la generacion de graficos estaticos

`Seaborn` = Para visualizacion de datos mas estilizada.

`streamlit` = Para crear interfaces interactivas con python.

`os` = Para gestion de archivos y directorios

`chardet` = Para caracteres especiales

In [ ]:
#----- Importación de Librerías -----------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import random
from skimage import io
import os
import chardet

### **Variables globales**

In [ ]:
LOGO_PATH = r'./media/images/MiBici_Logo.png'
LOGO_PATH_AGE = r'./media/images/Edad.png'
IMAGE_PATH_STATION = r'./media/images/Estacion.png'
GRAPH_PATH = r'./media/images/grafico.png'
#----- Configuracion datos -------------------------
DATA_FOLDER = r'data/MiBici-Data/Test'
DATA_NOMENCLATURA_FOLDER = r'data/Nomenclatura-Mibici-Data'
#----- Configuracion Cache -------------------------
CACHE_FOLDER = r'data/cache'
CACHE_FILE = os.path.join(CACHE_FOLDER, 'datos_procesados.parquet')

### **Deteccion de codificacion de archivos**
Antes de cargar los datos es importante procesar los archivos CSV, para poder conocer su codificacion para evitar errores de lectura. Esta funcion utiliza `chardet` para detectar la codificacion de un archivo.

`chardet.detect` = Para analizar el contenido del archivo y predecir su codificacion

`open(file, 'rb')` = Para leer el archivo en modo binario y evitar errores de codificacion

`raw_data.read()` = para cargar los datos del archivo en memoria

In [ ]:
#----- Funciones para decodificacion -----------------------------
def dectect_encoding(file):
    '''Funcionalidad para dectectar la codificacion de un archivo'''
    with open(file, 'rb') as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    return result['encoding']


### **Carga y procesamiento de Datos**
Se creo una funcion para cargar los datos de **MiBici** del formato .CSV, estandarizar nombres de columnas.

`pandas.read_csv` = para leer archivos CSV y convertirlos en dataFrame

`os.walk` = para recorrer directorios y listar archivos

`pandas.concat` = para combinar multiples DataFrames en uno solo

In [ ]:
#----- Funciones para leer y procesamiento de datos --------------
def cargar_datos(data_folder):
    '''Funcionalidad para leer y procesar datos de un archivo CSV'''
    # Formato de las columnas correctas
    columnas_correctas = [
        "Trip_Id", "User_Id", "Gender", "Year_of_Birth",
        "Trip_Start", "Trip_End", "Origin_Id", "Destination_Id"
    ]

    #Diccionario para renombrar columnas
    renombrar_columnas = {
        "Viaje_Id": "Trip_Id",
        "Usuario_Id": "User_Id",
        "Genero": "Gender",
        "Año_de_nacimiento": "Year_of_Birth",
        "A}äe_nacimiento": "Year_of_Birth",
        "AÃ±o_de_nacimiento": "Year_of_Birth",
        "Aï¿½o_de_nacimiento": "Year_of_Birth",
        "Inicio_del_viaje": "Trip_Start",
        "Fin_del_viaje": "Trip_End",
        "Origen_Id": "Origin_Id",
        "Destino_Id": "Destination_Id"
    }

    #Lista, almacenar todos los dataframe
    dataframes = []

    #Iterar todos los archivos y subcarpetas
    for root, _, files in os.walk(data_folder):
        for file in files:
            # leer archivo CSV
            if file.endswith(".csv"):
                ruta_completa = os.path.join(root, file)
                ruta_completa = ruta_completa.replace("\\", "/")
                #print(f"📂 Intentando leer: {ruta_completa}")

                try:
                    #Dectectar la codificacion del archivo
                    encoding = dectect_encoding(ruta_completa)
                    #print(f"🔍 Codificación detectada: {encoding}")

                    #leer archivo
                    df = pd.read_csv(ruta_completa, encoding=encoding)

                    if df is not None and not df.empty:
                        #print(f"✅ Archivo cargado correctamente: {ruta_completa}")

                        # Renombrar columnas
                        df = df.rename(columns=renombrar_columnas, inplace=False)
                        # Verificar si las columnas están en el orden correcto
                        df = df[[col for col in columnas_correctas if col in df.columns]]
                        #print(f"📊 Columnas renombradas y ordenadas correctamente {df.columns}")

                        #Extraer el año, mes  del archivo
                        nombre_archivo = os.path.basename(ruta_completa)
                        partes = nombre_archivo.split('_')
                        if len(partes) >= 4:
                            anio = int(partes[2])
                            mes = int(partes[3].split('.')[0])
                            df['Year'] = anio
                            df['Month'] = mes

                        #Agregar datos al dataframe
                        dataframes.append(df)
                        st.success(f'✅ Datos cargados: {ruta_completa} - Columnas: {df.columns.tolist()}')
                    else:
                        #print(f"⚠️ Archivo vacío o no se pudo leer: {ruta_completa}")
                        st.error(f'❌ No se pudo leer el archivo: {ruta_completa} esta vacio o no se pudo leer')

                except Exception as e:
                    #print(f"❌ Error leyendo {ruta_completa}: {e}")  # Debugging
                    st.error(f'❌ Error al leer el archivo {ruta_completa}: {e}')

    # Concatenar todos los datos del dataframe en 1
    if dataframes:
        all_data = pd.concat(dataframes, ignore_index=True)
        st.success('Datos cargados y unificados correctamente')
        return all_data
    else:
        st.error('No se pudieron concatenar los datos')
        return  None

### **Carga y procesamiento de Datos | Nomenclatura**
Se creo una funcion para cargar los datos de **MiBici** pero relacionado con la **nomenclatura** del formato .CSV.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~ Nomenclatura ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def cargar_nomenclatura(data_nomenclatura_folder):
    '''Funcionalidad para cargar los datos de Nomenclatura de MiBici'''
    nomenclatura_dataframes = []

    # Debug: Verificar si la carpeta existe
    #print(f'📂 Verificando si la carpeta de nomenclatura existe: {os.path.exists(data_nomenclatura_folder)}')

    #Cargar y procesar el archivo de nomenclatura
    for root, _, files in os.walk(data_nomenclatura_folder):
        #print(f'📁 Explorando: {root} - Archivos encontrados: {files}')
        for file in files:
            if file.endswith(".csv"):
                #Lectura de archivo
                ruta_completa = os.path.join(root, file)
                ruta_completa = ruta_completa.replace('\\','/')
                #print(f'📂 Intentando leer: {ruta_completa}')

                try:
                    #Dectectar codificador
                    encoding = dectect_encoding(ruta_completa)
                    #print(f'🔍 Codificacion dectectada: {encoding}')

                    #leer CSV
                    df = pd.read_csv(ruta_completa, encoding=encoding)
                    #print(f'✅ Archivo leido correctamente: {ruta_completa}')

                    if df is not None and not df.empty:
                        #añadir datos al dataframe
                        nomenclatura_dataframes.append(df)

                    else:
                        st.error(f'❌ Archivo vacio o no se pudo leer: {ruta_completa}')
                except Exception as e:
                    st.error(f'❌ Error al leer el archivo {ruta_completa}: {e}')
                    #print(f'🛑 Detalles del error: {str(e)}')

    if nomenclatura_dataframes:
        nomenclatura_df = pd.concat(nomenclatura_dataframes, ignore_index=True)
        #print(f' Dataframe final de nomenclatura: {nomenclatura_df.shape}')

        return nomenclatura_df
    else:
        st.error('No se pudieron cargar los datos de Nomenclatura')
        return None


### **Manejamiento de valores null/vacios**
Se elimina valores nulos en columnas criticas y se aplica `ffill` para completar datos faltantes. en este caso lo aplicamos para ambos archivos. datos de mibici y la nomenclatura.

---

`dropna` = Para eliminar valores nulos

`fillna` = Para rellenar valores nulos con datos previos.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~ Manejamiento de valores vacios/null ~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def manejar_valores_nulos_mibici(df):
    '''Funcionalidad para manejar valores nulos en el dataframe de "MiBici" '''
    if df is not None and not df.empty:
        #no pueden contener valores nulos
        columnas_criticas = ["Trip_Id", "User_Id", "Origin_Id", "Destination_Id"]

        #Eliminar filas con valor nulos
        df = df.dropna(subset=columnas_criticas)

        #Rellenar valores nulos
        df = df.ffill()

        st.success('Valores Nulos en datos de MiBici Manejados correctamente')
    return df

def manejar_valors_nulos_nomenclatura(df):
    '''Funcionalidad para manejar valores nulos en el dataframe de "Nomenclatura"'''
    if df is not None and not df.empty:
        #no pueden contener valores nulos
        columnas_criticas = ["id", "name", "latitude", "longitude"]

        # Eliminar filas con valor nulos
        df = df.dropna(subset=columnas_criticas)

        #Rellenar valores nulos
        df = df.ffill()

        st.success('Valores Nulos en datos de Nomenclatura Manejados correctamente')
    return df


### **Manejamiento de incosistencias.**
Las fechas de inicio y fin de viaje pueden tener incosistencias o valores no validos.
Por lo que:
`datetime` = Convierte las fechas a formato **datetime**
Elimina valores nulos e invalidos
Filtra viajes con horas irreales (12:00 y 15:00)
Se le da formato a las fechas `(YYYY - MM - DD HH:MM:SS)`

---

`pd.to_datetime` = Convierte cadenas a formato de texto y hora

`dropna(subnet = [...])`Elimina el registro con fecha nulas.

`strftime('%H:%M:%S)` = Extrae la hora de una fecha

`dt.strftime(%Y-%m-%d %H:%M:%S)` = formatea las fechas en un formato estandar.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~ Manejamiento incosistencias ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def manejar_fecha(df):
    '''Funcionalidad para manejar inconsistencia de fecha y hora inicio/fin '''
    if df is not None and not df.empty:
        # Convertir las columnas de fecha a datetime
        df['Trip_Start'] = pd.to_datetime(df['Trip_Start'], format='mixed', errors='coerce')
        df['Trip_End'] = pd.to_datetime(df['Trip_End'], format='mixed', errors='coerce')

        # Eliminar filas con fechas inválidas
        df = df.dropna(subset=['Trip_Start', 'Trip_End'])

        # Eliminar datos de prueba (horas perfectas (12 y 15))
        df = df[~(
            (df['Trip_Start'].dt.strftime('%H:%M:%S') == '12:00:00') & 
            (df['Trip_End'].dt.strftime('%H:%M:%S') == '15:00:00')
        )]

        # Asegurar el formato AAAA-MM-DD HH:MM:SS
        df['Trip_Start'] = df['Trip_Start'].dt.strftime('%Y-%m-%d %H:%M:%S')
        df['Trip_End'] = df['Trip_End'].dt.strftime('%Y-%m-%d %H:%M:%S')

        #print('Inconsistencias en fechas corregidas correctamente')
    return df

### **Calcular costos por viaje**
Se creo esta funcionalidad debido a que en un futuro se requiere esta funcion para mandar a llamar el calculo y poder hacer un grafico base a eso

---

Reglas:
- Costo individual al dia = `108 MXN`

- Costo extra de `29 MXN` despues de los primeros 30 minutos

- costo adicional de `40 MXN` por cada infraccion de 30 minutos extra minutos

- Costo extra de `29 MXN` despues de los primeros 30 minutos

In [ ]:
def calcular_costo(travel_time):
    '''Funcionalidad para calcular el costo de MiBici'''
    base_cost = 108
    extra_cost = 0

    total_minutes = travel_time.total_seconds() / 60
    if total_minutes > 30:
        extra_minutes = total_minutes - 30
        extra_cost += 29
        extra_minutes -= 30

        if extra_minutes > 0:
            extra_minutes += (extra_minutes // 30) * 40
            if extra_minutes % 30 > 0:
                extra_cost += 40

    return base_cost + extra_cost


### **Gestion de cache**
Se implemento una funcionalidad de cache, debido a que cada que ejecutabas el streamlit, tenias que recargar los datos (En mi programa) sin embargo se implemento una funcionalidad de cache para tener todos los datos en un solo archivo y de ahi mismo a hacer la lectura para todo sin necesidad de volver a recargar todo.

---

`save_cache` = Guarda un dataframe en cache

`load_cache` =carga un Dataframe desde la cache cache

---

`@st.cache_data` = Decorador de streamlit para almacenar datos en cache y evitar recoputaciones innecesarias

`df.to_parquet()` = Guarda un DataFrame en formato `.parquet` para almacenamiento eficientes

`pd.read_parquet()` = Carga un DataFrame desde un archivo `.parquet`

`os.path.exists()` = Verifica si el archivo de cache existe antes de intentar leerlo.

In [ ]:
#----- Cache  -----------------------------------
@st.cache_data
def save_cache(df, cache_file):
    '''Funcionalidad para guardar el Dataframe en un archivo cache'''
    if df is not None and not df.empty:
        #print(f"💾 Guardando datos en cache: {df.shape}")
        df.to_parquet(cache_file)

@st.cache_data
def load_cache(cache_file):
    '''Funcionalidad para guardar el Dataframe en un archivo cache'''
    if os.path.exists(cache_file):
        try:
            df = pd.read_parquet(cache_file)
            #print(f"✅ Archivo cache cargado correctamente: {df.shape}")
            return df
        except Exception as e:
            #print(f"❌ Error cargando archivo cache: {e}")
            return None
        #return pd.read_parquet(cache_file)
    else:
        #print("⚠️ Archivo cache no encontrado")
        return None


### **Agrupacion de Estaciones con Nomenclatura**
La funcion `estaciones()` permite asociar los identificadores de las estaciones (`Origin_Id` y `Destination_Id`) con sus nombres correspondientes en la nomenclatura. Esto facilita la interpretacion de los datos, ya que en lugar de los numeros de estacion, se presentan los nombres reales.

---

`merge(left_on, right_on, how='left')` = Realiza una union entre 2 DataFrames para agregar informacion relevante

`rename(columns={})` = Cambia los nombres de columnas para mejor claridad.

`drop(columns=[])` = Elimina columnas innecesarias tras la fusion

In [ ]:
#----- Funcionalidad para agrupar las estaciones ----------------------
def estaciones(df_mibici, df_nomenclatura):
    '''Funcionalidad para Agrupar (Origin_Id y Destination_Id) con las estaciones (nomenclatura (id))'''
    # Seleccionar columnas relevantes de nomenclatura
    df_nomenclatura = df_nomenclatura[['id', 'name']].rename(columns={'id': 'Station_Id', 'name': 'Station_Name'})

    # Unir datos de MiBici con nomenclatura para obtener el nombre de la estación de origen
    df_mibici = df_mibici.merge(df_nomenclatura, left_on='Origin_Id', right_on='Station_Id', how='left') \
                        .rename(columns={'Station_Name': 'Origin_Station'}) \
                        .drop(columns=['Station_Id'])

    # Unir datos de MiBici con nomenclatura para obtener el nombre de la estación de destino
    df_mibici = df_mibici.merge(df_nomenclatura, left_on='Destination_Id', right_on='Station_Id', how='left') \
                        .rename(columns={'Station_Name': 'Destination_Station'}) \
                        .drop(columns=['Station_Id'])

    return df_mibici[['Trip_Id', 'Origin_Id', 'Origin_Station', 'Destination_Id', 'Destination_Station']]


### **Conteo de Viajes por estacion**
Esta funcion cuenta con la cantidad de viajes que **salen de** o **llegan a** cada estacion y los asocia con su respectivo nombre de estacio. Esto permite identificar las estaciones mas utilizadas.

---

`value_counts()` = Cuenta la frecuencia de cada estacion en los viajes.

`reset_index()` = Convierte los conteos de un Dataframe estructurado.

`merge(left_on, right_on, how='left')` = Une los conteos con los nombres de las estaciones

`drop_duplicates()` = Elimina registros duplicados en la union de datos


In [ ]:
#----- Generar un conteo x Estacion ------------------------------
def conteo_estacion(df_mibici, df_nomenclatura, tipo):
    '''Funcionalidad para generar un conteo de viajes por estaciones'''
    # Cargar los datos
    df_agrupado = estaciones(df_mibici, df_nomenclatura)

    # Si el df esta vacio, regresar None
    if df_agrupado is None or df_agrupado.empty:
        return None

    if tipo == 'Salen':
        # Contar la cantidad de viajes por estación de salida
        conteo = df_agrupado['Origin_Id'].value_counts().reset_index()
        conteo.columns = ['Origin_Id', 'OutCount_Station']

        # Juntar el Id y contador con los nombres de las estaciones de origen
        conteo = conteo.merge(df_agrupado[['Origin_Id', 'Origin_Station']].drop_duplicates(), on='Origin_Id', how='left')
        return conteo[['Origin_Id', 'Origin_Station', 'OutCount_Station']]

    elif tipo == 'Llegan':
        # Contar la cantidad de viajes por estación de llegada
        conteo = df_agrupado['Destination_Id'].value_counts().reset_index()
        conteo.columns = ['Destination_Id', 'InCount_Station']

        # Juntar el Id y contador con los nombres de las estaciones de destino
        conteo = conteo.merge(df_agrupado[['Destination_Id', 'Destination_Station']].drop_duplicates(), on='Destination_Id', how='left')
        return conteo[['Destination_Id', 'Destination_Station', 'InCount_Station']]

    return None


### **Calculo de la edad del usuario**
Esta funcion calcula la edad del usuario basandose en su año de nacimiento `(Year_of_Birth)`. Se aplica validaciones para evitar errores y datos incoherentes.

---

`pd.to_numeric()` = Convertir los valores a formato numerico.

`isnull().all()` = Verifica si todos los valores de una columna son nulos.

`loc[]` = Filtra valores incorrectos en la columna `Age`

`pd.NA` = Maneja valores nulos correctamente en Pandas.


In [ ]:
#----- Funcionalidad para crear columna Edad ----------------------
def edad(df):
    '''Funcionalidad para añadir columna de la edad del usuario'''
    if 'Year_of_Birth' not in df.columns:
        st.error('❌ No se pudo calcular la edad porque la columna Year_of_Birth no se encuentra')
        return df

    df['Age'] = pd.to_numeric(df['Year_of_Birth'], errors='coerce') #Convertir numerico

    #Manejamiento de error de valores null
    if df['Year_of_Birth'].isnull().all():
        st.error('❌ No se puede calcular la edad porque todos los valores de "Year_of_Birth" son nulos.')
        return df

    # Funcion Calcular la edad
    df['Age'] = 2025 - df['Age']
    df.loc[(df['Age'] < 0) | (df['Age']> 150), 'Age'] = pd.NA # Filtrar valores erroneos (Limite excedido)
    return df


### **Calculo del tiempo de recorrido**
Esta funcion calcula el tiempo transcurrido entre la hora inicio `(Trip_Start)` y la hora de finalizacion `(Trip_End)` de un viaje y lo almacena en una nueva columna `Travel_Time`. Se asegura de manejar valores nulos y diferencias de tiempo mayores a 24 horas.

---

`pd.to_datetime()` = Convierte las columnas a formato de fecha y hora.

`apply(lambda x: str(x).split()[-1])` = Extrae el tiempo en HH:MM:SS

`if 'days' not in str(x)` = Maneja diferencias de mas de 24 horas correctamente

`st.error()` = Muestra errores en Streamlit si los datos son invalidos.


In [ ]:
#----- Funcionalidad para crear columna de Tiempo recorrido --------
def tiempo_recorrido(df):
    '''Funcionalidad para añadir columna de "Travel_Time" calcular el tiempo de (Trip_Start y Trip_End)'''
    if df is None or df.empty:
        st.error('El DataFrame esta vacio o no es valido')
        return None

    try:
        #Convertir formato tiempo
        df['Trip_Start'] = pd.to_datetime(df['Trip_Start'])
        df['Trip_End'] = pd.to_datetime(df['Trip_End'])

        #Calculo para distancia
        df['Travel_Time'] = (df['Trip_End'] - df['Trip_Start'])

        #Formatear la diferencia en formato HH:MM:SS
        df['Travel_Time'] = df['Travel_Time'].apply(
            lambda x:str(x).split()[-1]
            if 'days' not in str(x) else(x) # Manejar diferencias > 24 H
        )

        #st.success('Columna "Travel_Time" agregada correctamente')
        return df

    except Exception as e:
        st.error('❌ No se pudo calcular el tiempo de recorrido porque la columna "')
        return None


## **Grafica Lineal:**
### **Cantidad de viajes (Mes / Año)**
Esta funcion genera una grafica linea que muestra la cantidad de viajes por mes o por año, dependiendo de la opcion seleccionada.

---

`groupby & size` = Agrupar y contar la cantidad de viajes.

`sns.lineplot` = Creacion de grafico lineal

`plt.figure, plt.xlabel, plt.ylabel, plt.grid` = Configuracion del grafico

`streamlit` = Para mostrar la interfaz y resultados


In [ ]:
#===== Grafica Lineal ==== Cantidad de viajes * (Mes y año) ======
def graf_viaje(datos_filtrados, opcion_filtrado, year_selected, month_selected):
    ''' Grafica Lineal para contar la cantidad de viajes por mes y año'''
    try:
        # Validar datos no vengan vacios
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la grafica')
            return

        # Opcion
        if opcion_filtrado == 'Año x Meses':
            # Agrupar por mes y contar la cantidad de viajes
            viajes_count = datos_filtrados.groupby('Month').size().reset_index(name='Cantidad_Viajes')
            # Configuracion Grafica
            titulo = f'Cantidad de viajes por Meses del (Año: {year_selected})'
            x_label = 'Mes'
            x_values = viajes_count['Month']
        else:
            # Agrupar por año y contar la cantidad de viajes
            viajes_count = datos_filtrados.groupby('Year').size().reset_index(name='Cantidad_Viajes')
            # Configuracion Grafica
            titulo = f'Cantidad de viajes por Años del (Mes: {month_selected})'
            x_label = 'Año'
            x_values = viajes_count['Year']

        # Mostrar Datos de conteo de viajes
        st.markdown('#### 📊 Conteo de Viajes:')
        st.dataframe(viajes_count)

        #creacion de la grafica
        plt.figure(figsize=(10,6))
        sns.lineplot(data=viajes_count, x=x_values, y='Cantidad_Viajes', marker='o')
        plt.title(titulo, fontsize=16)
        plt.xlabel(x_label, fontsize=12)
        plt.ylabel('Cantidad de Viajes', fontsize=12)
        plt.grid(True)
        plt.tight_layout()

        #Mostrar grafico
        return st.pyplot(plt)
    except Exception as e:
        st.error('❌ No se pudo generar la gráfica de viajes')


## **Grafica Barras:**
### **Promedio de viaje (Dia * Semana)**
Grafica de barras que muestra el promedio de viajes de MiBici por dia de la semana, permitiendo filtrado por mes o año.

---

**Pandas** = para manipulación y transformación de datos ``(pd.to_datetime, value_counts, reindex)``

**Sns (Seaborn)** = para crear gráficos de barras `sns.barplot`

**Matplotlib** = para configuraciones de la gráfica `plt.figure, plt.xlabel, plt.ylabel, plt.xticks, plt.grid`

**Streamlit** = para mostrar la interfaz y resultados `st.error, st.markdown, st.dataframe, st.pyplot`


In [ ]:
#===== Grafica Barras ==== Promedio de viaje (dia * semana) ======
def graf_uso_semanal(datos_filtrados, opcion_filtrado, year_selected, month_selected):
    ''' Grafica para contar el uso de MiBici por semana'''
    try:
        #Validar datos no esten vacios
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la grafica')
            return
        #Convertir a datetime
        datos_filtrados['Trip_Start'] = pd.to_datetime(datos_filtrados['Trip_Start'])
        #Obtener el dia de la semana (0 = Lunes - 6 = Domingo)
        datos_filtrados['Day_Week'] = datos_filtrados['Trip_Start'].dt.dayofweek
        #Diccionario para mapear el # a dias
        dias_semana = {0: 'Lunes', 1: 'Martes', 2: 'Miercoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sabado', 6: 'Domingo',}
        #Remplazar los valores numericos a nombre del dia
        datos_filtrados['Day_Week'] = datos_filtrados['Day_Week'].map(dias_semana)
        #Conteo de # viajes por cada dia de la semana
        count_viajes = datos_filtrados['Day_Week'].value_counts().reindex(dias_semana.values(), fill_value = 0)

        #Opcion filtrado [Año x Meses | Mes x Año]
        if opcion_filtrado == "Año x Meses":
            titulo = f'Uso de MiBici por Día de la Semana - Año {year_selected}'
            subtitulo = f'Meses seleccionados: {month_selected}'
        elif opcion_filtrado == "Mes x Años":
            titulo = f'Uso de MiBici por Día de la Semana - Mes {month_selected}'
            subtitulo = f'Años seleccionados: {year_selected}'
        else:
            titulo = 'Uso de MiBici por Día de la Semana'
            subtitulo = ''

        # Mostrar tabla de conteo
        st.markdown(f'#### 📊 {titulo}:')
        if subtitulo:
            st.markdown(f'{subtitulo}')
        st.dataframe(count_viajes.reset_index().rename(columns={'index': 'Day', 'Day_Week': 'Cantidad de Viajes'}))

        #Creacion y Configuracion Grafica
        plt.figure(figsize=(10,6))
        sns.barplot(x=count_viajes.index, y =count_viajes.values, hue=count_viajes.index, palette='coolwarm', legend=False)

        # conf.
        plt.title(f'{titulo}', fontsize=16)
        plt.xlabel('Dia de la semmana', fontsize=12)
        plt.ylabel('Cantidad de Viajes', fontsize=12)
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        #Mostrar grafico
        st.pyplot(plt)

    except Exception as e:
        st.error(f'❌ No se pudo generar la gráfica de uso semanal {e}')


## **Grafica Barras:**
### **Uso de MiBici por dia**
Grafica de barras para visualizar el uso de MiBici por dia del mes permitiendo analizar la distribuccion de viajes en diferentes periodos de tiempo.


In [ ]:
#===== Grafica Barras ==== Uso por Dias de MiBici ================
def graf_dias(datos_filtrados, opcion_filtrado, year_selected, month_selected):
    '''Grafica para mostrar el uso de MiBici en los dias.'''
    try:
        #Validar que los datos no esten vacios
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la grafica.')
            return

        #Extraer el dia del mes
        datos_filtrados['Day'] = datos_filtrados['Trip_Start'].dt.day

        #Contar Viajes por dia y aplicar filtros
        if opcion_filtrado == 'Año x Meses':
            #Agrupar por Año-Mes-Dia
            count_days = datos_filtrados.groupby(['Year', 'Month', 'Day']).size().reset_index(name='Count')
            # Convertir Month a string para visualización clara
            count_days['Month'] = count_days['Month'].astype(str)

        else:
            # Agrupar por Año-Mes-Día
            count_days = datos_filtrados.groupby(['Year', 'Month', 'Day']).size().reset_index(name='Count')
            # Convertir Year a string para visualizarlo correctamente
            count_days['Year'] = count_days['Year'].astype(str)

        st.markdown('#### 📊 Conteo de Viajes por Día:')
        st.dataframe(count_days)

        # Creacion y conf la grafica
        plt.figure(figsize=(12,6))
        sns.set_style('whitegrid')

        if opcion_filtrado == 'Año x Meses':
            # Grafico varios meses del mismo año
            sns.lineplot(data=count_days, x = 'Day', y='Count', hue='Month', markers='o', palette='tab10')
            plt.title(f'Cantidad de Viajes por Día - Año: {year_selected}', fontsize=14)
            plt.xlabel('Día del Mes', fontsize=12)
            plt.ylabel('Cantidad de Viajes', fontsize=12)
            plt.legend(title="Meses")

        else:
            # Graficar múltiples años del mismo mes con distintos colores
            sns.lineplot(data=count_days, x='Day', y='Count', hue='Year', marker='o', palette='tab10')
            plt.title(f'Cantidad de Viajes por Día en el Mes: {month_selected}', fontsize=14)
            plt.xlabel('Día del Mes', fontsize=12)
            plt.ylabel('Cantidad de Viajes', fontsize=12)
            plt.legend(title="Años")

        # Mostrar la gráfica en Streamlit
        st.pyplot(plt)

    except Exception as e:
        st.error(f'❌ No se pudo generar la grafica {str(e)}')


## **Grafica Histograma:**
### **Hombres vs Mujeres Uso MiBici**
Muestra un histograma que compara el uso de MiBici entre hombres y mujeres a lo largo de la semana. Se agrupan los datos por el dia de la semana y genero, permitiendo visualizar que dias tienen mayor uso por parte de cada genero.
La funcion incluye validaciones para evitar errores con datos vacios y muestra la informacion en una tabla antes de graficarla.


In [ ]:
#===== Grafica Histograma === Hombres vs Mujeres = Uso de MiBici =
def graf_gender_versus(datos_filtrados, opcion_filtrado, year_selected, month_selected):
    '''Grafica para mostrar la comparativa de H vs M al usar MiBici durante la semana'''
    try:
        #Validar que los datos no esten
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la grafica')
            return
        # Convertir a Trip_Start a datetime
        datos_filtrados['Trip_Start'] = pd.to_datetime(datos_filtrados['Trip_Start'])
        #Obtener el dia de la semana (0 = Lunes - 6 = Domingo)
        datos_filtrados['Day_Week'] = datos_filtrados['Trip_Start'].dt.dayofweek
        #Diccionario para mapear el # a dias
        dias_semana = {0: 'Lunes', 1: 'Martes', 2: 'Miercoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sabado', 6: 'Domingo',}
        #Remplazar los valores numericos a nombre del dia
        datos_filtrados['Day_Week'] = datos_filtrados['Day_Week'].map(dias_semana)
        #Definir el orden correcto de los dias de la semana
        orden_dias = ['Lunes','Martes','Miercoles','Jueves', 'Viernes','Sabado','Domingo',]
        datos_filtrados['Day_Week'] = pd.Categorical(datos_filtrados['Day_Week'],categories = orden_dias, ordered=True)
        #Conteo de # viajes por genero durante la semana dia de la semana
        count_gender = datos_filtrados.groupby(['Day_Week', 'Gender'], observed=False).size().reset_index(name='Cantidad_Viajes')

        # Pivotear la tabla para que cada género sea una columna
        count_pivot = count_gender.pivot(index='Day_Week', columns='Gender', values='Cantidad_Viajes').fillna(0)

        #Opcion filtrado [Año x Meses | Mes x Año]
        if opcion_filtrado == "Año x Meses":
            titulo = f'Uso de MiBici por Día de la Semana - Año {year_selected}'
            subtitulo = f'Meses seleccionados: {month_selected}'
        elif opcion_filtrado == "Mes x Años":
            titulo = f'Uso de MiBici por Día de la Semana - Mes {month_selected}'
            subtitulo = f'Años seleccionados: {year_selected}'
        else:
            titulo = 'Uso de MiBici por Día de la Semana'
            subtitulo = ''

        # Mostrar la tabla
        st.markdown(f'#### 📊 {titulo}')
        if subtitulo:
            st.markdown(f'{subtitulo}')
        st.dataframe(count_pivot)

        # Crear la gráfica
        plt.figure(figsize=(10,6))
        colores = ['#ff69b4', '#1f77b4']
        count_pivot.plot(kind='bar', stacked=False, color=colores, alpha=0.8, width=0.8)

        # Configuración del gráfico
        plt.title(f'Comparativa H vs M de {titulo} ', fontsize=16)
        plt.xlabel('Día de la Semana', fontsize=12)
        plt.ylabel('Cantidad de Viajes', fontsize=12)
        plt.xticks(rotation=45)
        plt.legend(title='Genero', labels=['Mujeres', 'Hombres'])
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        # Mostrar gráfico en Streamlit
        st.pyplot(plt)

    except Exception as e:
        st.error(f'❌ No se pudo generar la grafica')


## **Correlacion:**
### **Correlacion dia de las semanas**
Muestra un histograma que compara el uso de MiBici entre hombres y mujeres a lo largo de la semana. Se agrupan los datos por el dia de la semana y genero, permitiendo visualizar que dias tienen mayor uso por parte de cada genero.
La funcion incluye validaciones para evitar errores con datos vacios y muestra la informacion en una tabla antes de graficarla.


In [ ]:
#===== Grafico Correlacion ==== Correlacion Dia de la semanas ====
def graf_dia_time(datos_filtrados):
    try:
        #Validar que los datos no esten
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la grafica')
            return

        # Llamar a la funcion para extraer el 'Travel_Time'
        datos_filtrados = tiempo_recorrido(datos_filtrados)

        # Validacion de que exista la columna "Travel_Time"
        if 'Travel_Time' not in datos_filtrados.columns:
            st.error('❌ La columna Travel_Time No existe, no se puede crear la funcion')
            return

        # Debbugin
        #st.write("Primeros 5 valores de Travel_Time:")
        #st.write(datos_filtrados[['Trip_Id', 'Trip_Start', 'Trip_End', 'Travel_Time']].head(15))

        #Convertir 'Travel_time' a timedelta
        if not pd.api.types.is_timedelta64_dtype(datos_filtrados['Travel_Time']):
            datos_filtrados['Travel_Time'] = pd.to_timedelta(datos_filtrados['Travel_Time'])

        #Diccionario para mapear el # a dias
        dias_semana = {0: 'Lunes', 1: 'Martes', 2: 'Miercoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sabado', 6: 'Domingo',}
        datos_filtrados['Day_Week'] = datos_filtrados['Trip_Start'].dt.weekday.map(dias_semana)

#        #Calcular promedio de tiempo de viaje por dia de la semana
#        promedio_viajes = datos_filtrados.groupby('Day_Week')['Travel_Time'].mean().reset_index()
#        promedio_viajes['Travel_Time'] = promedio_viajes['Travel_Time'].dt.total_seconds() / 60  # Convertir a minutos

        #Ordenar dias de la semana
        orden_dias = ['Lunes','Martes','Miercoles','Jueves', 'Viernes','Sabado','Domingo',]
        datos_filtrados['Day_Week'] = pd.Categorical(datos_filtrados['Day_Week'],categories = orden_dias, ordered=True)
        datos_filtrados = datos_filtrados.sort_values('Day_Week')

        #Convertir Travel_Time a minutos
        datos_filtrados['Travel_Time_Minutos'] = datos_filtrados['Travel_Time'].dt.total_seconds() / 60

        #Calcular los percentiles
        y_min = datos_filtrados['Travel_Time_Minutos'].quantile(0.0) #percentil 0
        y_max = datos_filtrados['Travel_Time_Minutos'].quantile(0.99) #percentil 99

        # --- Gráfico de Dispersión con Línea de Tendencia ---
        fig, ax = plt.subplots(figsize=(10, 5))

        # Mapear los días de la semana a números para el eje X
        datos_filtrados['Day_Week_Num'] = datos_filtrados['Day_Week'].map({dia: i for i, dia in enumerate(orden_dias)})

        # Scatter plot
        sns.scatterplot(
            x='Day_Week_Num', 
            y='Travel_Time_Minutos', 
            data=datos_filtrados, 
            color='blue', 
            alpha=0.6, 
            ax=ax
        )

        # Línea de tendencia
        sns.regplot(
            x='Day_Week_Num', 
            y='Travel_Time_Minutos', 
            data=datos_filtrados, 
            scatter=False, 
            color='red', 
            ax=ax
        )

        #calcular el promedio de "Travel_Time_Minutos" por dia de la semana
        promedio_viajes = datos_filtrados.groupby('Day_Week')['Travel_Time_Minutos'].mean().reset_index()

        # Mostrar el promedio como una línea horizontal
        for dia, promedio in zip(promedio_viajes['Day_Week'], promedio_viajes['Travel_Time_Minutos']):
            ax.axhline(promedio, color='green', linestyle='--', alpha=0.5, label=f'Promedio {dia}: {promedio:.2f} min')

        # Configuración del gráfico
        ax.set_title("Correlación Día de la Semana - Tiempo de Viaje", fontsize=16)
        ax.set_xlabel("Día de la Semana", fontsize=12)
        ax.set_ylabel("Tiempo de Viaje (minutos)", fontsize=12)
        ax.set_xticks(range(len(orden_dias)))
        ax.set_xticklabels(orden_dias, rotation=45)
        ax.set_ylim(y_min, y_max)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        # Mostrar gráfico en Streamlit
        st.pyplot(fig)

        # --- Boxplot para ver distribución ---
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.boxplot(
            [datos_filtrados[datos_filtrados['Day_Week'] == dia]['Travel_Time_Minutos'] for dia in orden_dias],
            labels=orden_dias
        )
        ax.set_title("Distribución del Tiempo de Viaje por Día de la Semana", fontsize=16)
        ax.set_xlabel("Día de la Semana", fontsize=12)
        ax.set_ylabel("Tiempo de Viaje (minutos)", fontsize=12)
        ax.set_ylim(y_min, y_max)
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        # Mostrar gráfico en Streamlit
        st.pyplot(fig)


    except Exception as e:
        st.error(f'❌ No se pudo generar la grafica {str(e)}')


## **Correlacion / Dispersion:**
### **Edad y Tiempo promedio**
Genera un grafico de dispersion que muestra la relacion entre la edad de los usuarios y el tiempo promedio de viaje en MiBici primero, valida que los datos no esten vacios y calcula la edad y la duracion de los viajes en minutos. luego lo agrupa con los datos de la edad, filtra los valores fuera de un rango logico (16-120 años).


In [ ]:
#===== Grafica Correlacion ==== Edad - Tiempo promedio ==========
def graf_edad_time(datos_filtrados, opcion_filtrado, year_selected, month_selected):
    '''Grafica para mostrar la edad con tiempo promedio'''
    try:
        # Validar que los datos no estén vacíos
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la gráfica.')
            return

        #Calcular edad usuarios
        datos_filtrados = edad(datos_filtrados)

        #Calcular tiempo de viaje
        datos_filtrados = tiempo_recorrido(datos_filtrados)

        #Verificar que las columnas necesarias esten presentes
        if 'Age' not in datos_filtrados.columns or 'Travel_Time' not in datos_filtrados.columns:
            st.error('❌ No se pudieron calcular las columnas necesarias (Age o Travel_Time).')
            return

        #convertir Travel_Time a minutos
        datos_filtrados['Travel_Time'] = pd.to_timedelta(datos_filtrados['Travel_Time'])
        datos_filtrados['Travel_Time_Minutos'] = datos_filtrados['Travel_Time'].dt.total_seconds() / 60

        #Agrupar por edad y calcular el tiempo promedio de viaje
        tiempo_promedio_edad = datos_filtrados.groupby('Age')['Travel_Time_Minutos'].mean().reset_index()

        #Filtrar edades validas
        tiempo_promedio_edad = tiempo_promedio_edad[(tiempo_promedio_edad['Age'] >= 16) & (tiempo_promedio_edad['Age'] <= 120)]
        #Ajustar el Eje y
        y_max = tiempo_promedio_edad['Travel_Time_Minutos'].quantile(0.99) #percentil 99

        #Opcion filtrado [Año x Meses | Mes x Año]
        if opcion_filtrado == "Año x Meses":
            titulo = f'Correlacion Edad - Tiempo promedio de viaje. - Año {year_selected}'
            subtitulo = f'Meses seleccionados: {month_selected}'
        elif opcion_filtrado == "Mes x Años":
            titulo = f'Correlacion Edad - Tiempo promedio de viaje. - Mes {month_selected}'
            subtitulo = f'Años seleccionados: {year_selected}'
        else:
            titulo = 'Correlacion Edad - Tiempo promedio de viaje.'
            subtitulo = ''

        # Mostrar la tabla
        st.markdown(f'#### 📊 {titulo}')
        if subtitulo:
            st.markdown(f'{subtitulo}')
        st.dataframe(tiempo_promedio_edad)

        #Creacion de grafico
        fig, ax = plt.subplots(figsize = (12, 6))

        #ScatterPlot
        sns.scatterplot(
            x="Age",
            y="Travel_Time_Minutos",
            data=tiempo_promedio_edad,
            color='blue',
            alpha=0.6,
            ax=ax
        )

        #Linea de tendencia
        sns.regplot(
            x="Age",
            y="Travel_Time_Minutos",
            data=tiempo_promedio_edad,
            scatter=False,
            color='red',
            ax=ax
        )

        # conf
        ax.set_title(f'{titulo}', fontsize =16)
        ax.set_xlabel('Edad (>= 16) ', fontsize=12)
        ax.set_ylabel('Tiempo Promedio de viaje (Minutos)', fontsize=12)
        ax.set_ylim(0, y_max)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        #Mostrar grafico
        st.pyplot(fig)

    except Exception as e:
        st.error(f'❌ No se pudo generar la gráfica. Error: {str(e)}')


## **Grafica de barras:**
### **Uso por estaciones**
Genera una grafica de barras para mostrar el uso de estaciones en base a un tipo especifico de conteo (viajes de salida o llegada). Recibe datos filtrados y un Dataframe de nomenclatura y cuenta el numero de viajes por estacion. .


In [ ]:
#===== Grafica Barras === Uso de estaciones  ====================
def graf_use_station(datos_filtrados, nomenclatura_df, tipo):
    '''Grafica para mostrar el uso de cada estacion'''
    try:
        # Validar que los datos no estén vacíos
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la gráfica.')
            return

        #Obtener el conteo de viajes por estacion
        count = conteo_estacion(datos_filtrados, nomenclatura_df, tipo)

        # Validar que el conteo no se encuentre vacio
        if count is None or count.empty:
            st.error('❌ No se pudo generar el conteo de estaciones.')
            return

        #Extraer el identificador de c/Estacion
        if tipo == 'Salen':
            count['Station_Code'] = count['Origin_Station'].str.extract(r'\((.*?)\)')
            x_label = 'Estacion de Salida'
            y_label = 'Conteo de Viajes de Salida'
        elif tipo == 'Llegan':
            count['Station_Code'] = count['Destination_Station'].str.extract(r'\((.*?)\)')
            x_label = 'Estacion de Llegada'
            y_label = 'Conteo de Viajes de Llegada'
        else:
            st.error('❌ Tipo de conteo no valido. Usa "Salen" o "Llegan".')
            return

        #Ordenar por conteo de viajes
        count = count.sort_values(by=count.columns[2], ascending= False)

        # Creacion de grafico y configuracion

        # --- Grafico de barras ---
        plt.figure(figsize=(12,6))
        sns.barplot(
            x = 'Station_Code',
            y = count.columns[2],
            data = count,
            palette='coolwarm'
        )

        # conf grafico
        plt.title(f' Uso de Estaciones ({tipo})', fontsize=16)
        plt.xlabel(x_label, fontsize=12)
        plt.ylabel(y_label, fontsize=12)
        plt.xticks(rotation=90)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        #Mostrar grafico
        st.pyplot(plt)

    except Exception as e:
        st.error(f'❌ No se pudo generar la gráfica. Error: {str(e)}')


## **Grafica de barras:**
### **Total de dinero Gastado**
Grafica de barras que muestra el total de dinero gastado en funcion de los viajes, agrupados por dia de la semana. La funcion toma datos filtrados y filtra por año o mes segun la opcion que ha sido seleccionada. Calcula el costo de cada viaje en funcion del tiempo de viaje y lo agrupo por dia, sumando tel total de cada dia de la semana.

In [ ]:
#===== Grafica barras ==== Total de dinero gastado (aproximado) =
def graf_money(datos_filtrados, opcion_filtrado, year_selected, month_selected):
    '''Grafica para mostrar el total de dinero gastado'''
    try:
        # Validar que los datos no estén vacíos
        if datos_filtrados is None or datos_filtrados.empty:
            st.error('❌ No hay datos filtrados para generar la gráfica.')
            return

        # Convertir Trip_Start a datetime si no está en formato datetime
        datos_filtrados['Trip_Start'] = pd.to_datetime(datos_filtrados['Trip_Start'], errors='coerce')

        # Extraer el día de la semana en español
        dias_semana = {
            'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miercoles',
            'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sabado', 'Sunday': 'Domingo'
        }
        datos_filtrados['Day'] = datos_filtrados['Trip_Start'].dt.day_name().map(dias_semana)

        # Convertir Travel_Time a minutos si es timedelta
        if pd.api.types.is_timedelta64_dtype(datos_filtrados['Travel_Time']):
            datos_filtrados['Travel_Time_Minutos'] = datos_filtrados['Travel_Time'].dt.total_seconds() / 60
        else:
            st.error('❌ Error: Travel_Time no está en formato timedelta.')
            return

        # Calcular costo
        datos_filtrados['Cost'] = datos_filtrados['Travel_Time'].apply(calcular_costo)

        # Agrupar por día de la semana y sumar el costo total
        costo_por_dia = datos_filtrados.groupby('Day', observed=True)['Cost'].sum().reset_index()

        # Ordenar días de la semana correctamente
        dias_ordenados = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
        costo_por_dia['Day'] = pd.Categorical(costo_por_dia['Day'], categories=dias_ordenados, ordered=True)
        costo_por_dia = costo_por_dia.sort_values('Day')

        #Opcion filtrado [Año x Meses | Mes x Año]
        if opcion_filtrado == "Año x Meses":
            titulo = f'Gasto total por dia de la semana - Año {year_selected}'
            subtitulo = f'Meses seleccionados: {month_selected}'
        elif opcion_filtrado == "Mes x Años":
            titulo = f'Gasto total por dia de la semana - Mes {month_selected}'
            subtitulo = f'Años seleccionados: {year_selected}'
        else:
            titulo = 'Gasto total por dia de la semana'
            subtitulo = ''

        # Mostrar la tabla
        st.markdown(f'#### 📊 {titulo}')
        if subtitulo:
            st.markdown(f'{subtitulo}')
        st.dataframe(costo_por_dia)

        # --- Gráfico de barras ---
        plt.figure(figsize=(10, 5))
        sns.barplot(
            x='Day',
            y='Cost',
            data=costo_por_dia,
            palette='summer'
        )

        # Configuración del gráfico
        plt.title(f'{titulo}', fontsize=14)
        plt.xlabel('Día de la semana', fontsize=12)
        plt.ylabel('Costo Total (MXN) Gastado', fontsize=12)
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()

        # Mostrar gráfico
        st.pyplot(plt)

    except Exception as e:
        st.error(f'❌ No se pudo generar la gráfica. Error: {str(e)}')
